In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [ ]:
# 데이터 다운 & 전처리

df = pd.read_excel('/content/더미데이터.xlsx')

In [ ]:
def to_list(x):
  x = x.replace('[','').replace(']','').replace('  ','').split(' ')
  return np.array(x,dtype=np.float64)

In [ ]:
df[0] = df[0].apply(to_list)
df[1] = df[1].apply(to_list)

In [ ]:
#### 문제가 있어!!!!!!!!!!!!!!!

# 필요하다면 파라미터 개수를 늘려
class windowDataset(Dataset):
    def __init__(self, data):

        data_len = len(data)
        X, Y =[],[]
        for j in range(data_len):
          X.append(data[0].values[j])

        for j in range(data_len):
          Y.append(df[1].values[j])

        self.x = torch.FloatTensor(X)
        self.y = torch.FloatTensor(Y)

        self.len = len(X)

    # return 값들
    def __getitem__(self, i):
        return self.x[i], self.y[i]

    def __len__(self):
        return self.len

In [ ]:
train_dataset = windowDataset(df)
train_loader = DataLoader(train_dataset, batch_size=64)

In [ ]:
for x,y in train_loader:
  print("X 크기 : {}".format(x))
  break

X 크기 : tensor([[    0.,     0., 59500.,     0.,     0.,     0.,     0.],
        [    0.,     0., 59500.,     0.,     0.,     0.,     0.],
        [    0.,     0., 59500.,     0.,     0.,     0.,     0.],
        [    0.,     0., 59500.,     0.,     0.,     0.,     0.],
        [    0.,     0., 59500.,     0.,     0.,     0.,     0.],
        [    0.,     0., 59500.,     0.,     0.,     0.,     0.],
        [    0.,     0., 60000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 60000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 60000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 60000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 60000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 67000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 67000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 67000.,     0.,     0.,     0.,     0.],
        [    0.,     0., 67000.,     0.,     0.,     0.,     0.],
   

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size=7):
        super(Model, self).__init__()

        # 인코더(encoder) 정의
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU()
        )

        # 디코더(decoder) 정의
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.ReLU(),
            nn.Linear(4, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = Model().to(device)


print('작동하는지 실험')
basic_data = torch.rand((1,7))
model(basic_data)


cpu is available
작동하는지 실험


tensor([[0.4597, 0.6331, 0.4248, 0.5375, 0.5385, 0.4315, 0.4162]],
       grad_fn=<SigmoidBackward0>)

In [ ]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
## 학습하는 코드를 짜봐!!!

train_dataset = windowDataset(df)
train_loader = DataLoader(train_dataset, batch_size=2, drop_last=True)
val_dataset = windowDataset(df)
val_loader = DataLoader(val_dataset, batch_size=2, drop_last=True)
test_dataset = windowDataset(df)
test_loader = DataLoader(test_dataset, batch_size=2, drop_last=True)


train_n = len(train_loader)
val_n = len(val_loader)

# print(train_loader)
best_val_loss = np.inf

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for inputs, labels in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        inputs = inputs.to(device)
        labels = labels.to(device)
        out = model(inputs)
        loss = criterion(out, labels)

        optimizer.zero_grad() #
        loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss.item() # 한 배치의 loss 더해주고,


    print(running_loss/train_n)

    with torch.no_grad():
              print("Calculating validation results...")
              model.eval()
              val_loss = 0
              for inputs, labels in val_loader:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outs = model(inputs)

                  loss= criterion(outs, labels)
                  val_loss += loss.item()



              # if val_acc > best_val_acc:
              #     print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
              #     # torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
              #     best_val_acc = val_acc
              # torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
              print("val")
              print(
                  f"loss: {val_loss/val_n:4.8} || "
              )
    print('======================================================')



epoch : 0
train
718056588.8
Calculating validation results...
val
loss: 7.1805659e+08 || 
epoch : 1
train
718056588.8
Calculating validation results...
val
loss: 7.1805659e+08 || 
epoch : 2
train
718056588.8
Calculating validation results...
val
loss: 7.1805659e+08 || 
epoch : 3
train
718056588.8
Calculating validation results...
val
loss: 7.1805659e+08 || 
epoch : 4
train
718056588.8
Calculating validation results...
val
loss: 7.1805659e+08 || 
